<a href="https://colab.research.google.com/github/Computer-CGuy/BitcoinTPU/blob/main/JAX_TPU_%7C_Old_Songs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
use_tpu = False
if(use_tpu):
    import jax.tools.colab_tpu
    jax.tools.colab_tpu.setup_tpu()
    import jax
    print(jax.devices())

In [3]:
import numpy as np

# We only need to import JAX's xla_client, not all of JAX.
from jax.lib import xla_client as xc
from jax.lib import xla_bridge as xb
xops = xc.ops

In [4]:
def ROL64(a,offset):
    return xops.Xor(xops.ShiftLeft(a,offset[0]),xops.ShiftRightArithmetic(a,offset[1]))

In [5]:
consts = [7,18,3,17,19,2,13,22,6,11,25]#[18,14],[3,29],[17,8],[19,13],[2,30],[13,19],[22,10],[6,26],[11,21],[25,7]]

In [6]:
converted = {}
def getconsts(num):
    global dtype
    global converted
    
    try:
        return converted[num]
    except:
        converted[num] = xops.Constant(miner, dtype(num))
    return converted[num]

In [7]:
def _rotr(x, const):
    offset_0 = getconsts(const)
    offset_1 = getconsts(32-const)
    return xops.Or(xops.ShiftLeft(x,offset_0),xops.ShiftRightArithmetic(x,offset_1))

def _maj(x, y, z):

    return xops.Xor(xops.And(x,y),xops.Xor(xops.And(x,y),xops.And(y,z)))
    # return (x & y) ^ (x & z) ^ (y & z)

def _ch(x, y, z):
    return xops.Xor(xops.And(x,y),xops.And(xops.Not(x),z))
    # return (x & y) ^ ((~x) & z)

In [8]:
def Xor3D(a,b,c):
    return xops.Xor(a,xops.Xor(b,c))

In [9]:
def listSum(l):
    sum = l[0]
    for x in range(1,len(l)):
        sum = xops.Add(sum,l[x])
    return sum

In [10]:
# Data Structure
## 64 UINT32 of W
## 8 UINT32 of a,b,...h

In [11]:
dtype = np.uint32
special_dtype = np.uint64

In [12]:
digit = xc.Shape.array_shape(np.dtype(special_dtype),())

In [13]:

def generator():
    w = [xc.Shape.array_shape(np.dtype(dtype),(batch,)) for x in range(64)]
    rolls = [xc.Shape.array_shape(np.dtype(dtype),(batch,)) for x in range(8)]
    # in_tuple_shape = xc.Shape.tuple_shape(w+rolls)
    return [digit]+w+rolls

In [14]:
# def inputer(types):
#     [for x in range(types)]

In [15]:
def degenerator(tup):
    w = [xops.GetTupleElement(tup,x+1) for x in range(64)]
    rolls = [xops.GetTupleElement(tup,x+64+1) for x in range(8)]
    return w,rolls

In [16]:
k = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
      0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
      0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
      0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
      0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
      0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
      0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
      0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
      0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
      0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
      0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
      0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
      0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
      0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
      0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
      0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]

In [17]:

def SHA256_hash(w,rolls,consts):
    a,b,c,d,e,f,g,h = rolls
    for i in range(16, 64):
        t1 = Xor3D(_rotr(w[i-15], consts[0]),_rotr(w[i-15], consts[1]),_rotr(w[i-15], consts[2]))
        # t1 = _rotr(w[i-15], consts[0])^_rotr(w[i-15], consts[1])^_rotr(w[i-15], consts[2])
        t2 = Xor3D(_rotr(w[i-2], consts[3]),_rotr(w[i-2], consts[4]),_rotr(w[i-15], consts[3]))
        # t2 = _rotr(w[i-2], consts[3])^_rotr(w[i-2], consts[4])^_rotr(w[i-15], consts[3])

        w[i] = xops.Add(w[i-16],xops.Add(t1,xops.Add(w[i-7],t2)))#w[i-16] + t1 + w[i-7]+t2
        
    for i in range(64): 
        
        t2 = Xor3D(_rotr(a, consts[5]),_rotr(a, consts[6]),_rotr(a, consts[7]))
        # torch.bitwise_xor(_rotr(a, consts[5]), _rotr(a, consts[6]),out=t2)
        # torch.bitwise_xor(t2,_rotr(a, consts[7]),out=t2)

        # s0[:] = _rotr(chrs[i%8], 2) ^ _rotr(chrs[i%8], 13) ^ _rotr(chrs[i%8], 22)
        t2 = xops.Add(t2 ,_maj(a, b, c))

        s1 = (Xor3D(_rotr(e, consts[8]),_rotr(e, consts[9]),_rotr(e, consts[10])))
        # torch.bitwise_xor(_rotr(e, consts[8]), _rotr(e, consts[9]),out=t1)
        # torch.bitwise_xor(t1,_rotr(e, consts[10]),out=t1)
        t1 = listSum([h,t1,_ch(e,f,g),getconsts(k[i]),t1,w[i]])
        # s1[:] = _rotr(chrs[(i+4)%8], 6) ^ _rotr(chrs[(i+4)%8], 11) ^ _rotr(chrs[(i+4)%8], 25)
        # torch.add(h,t1,out=t1)
        # torch.add(_ch(e,f,g) + k[i],t1+w[i],out=t1)
        # t1[:] = chrs[(i+7)%8] + s1 + _ch(chrs[(i+4)%8], chrs[(i+5)%8], chrs[(i+6)%8]) + k[i] + w[i]
        
        h = g
        g = f
        f = e
        e = xops.Add(d, t1)
        # torch.bitwise_and(e, F32,out=e)
        d = c
        c = b
        b = a
        a = xops.Add(t1,t2)
        # torch.add(t1, t2,out=a)
        # xm.mark_step()
        # torch.bitwise_and(a, F32,out=a)
        # a = (t1 + t2) & F32
    return w+[a,b,c,d,e,f,g,h]

In [18]:
# type(in_tuple_shape[0])

In [38]:
miner = xc.XlaBuilder("Main Function")
converted = {}
batch = 1

num = xc.Shape.array_shape(np.dtype(dtype),(batch,))
in_tuple_shape = generator()
in_tuple_shape = xc.Shape.tuple_shape(in_tuple_shape)
tup = xops.Parameter(miner,0,in_tuple_shape)

counter_ = xops.GetTupleElement(tup,0)
counter_ = xops.Add(counter_,xops.Constant(miner, special_dtype(1)))

w,rolls = degenerator(tup)

ans = SHA256_hash(w,rolls,consts)
ans = [counter_]+ans

output = xops.Tuple(miner,ans)

body_computation = miner.build()
# run(miner,batch=batch)

In [29]:

def run(c,batch=1,build=None):
    if(use_tpu):
        cpu_backend = xb._get_tpu_driver_backend("tpu")
    else:
        cpu_backend = xc.get_local_backend("cpu")
    if(not build):
        computation = c.Build()
    else:
        computation = build
    
    compiled_computation = cpu_backend.compile(computation)

    As = np.random.randint(0,0xFFFFFFFF-1,(64+8,batch),dtype=dtype)
    # state = [cpu_backend.buffer_from_pyval(dtype(0)),cpu_backend.buffer_from_pyval(dtype(ROUNDS))]
    state = [cpu_backend.buffer_from_pyval(special_dtype(0))]
    print("Evalving")
    for x in range(64+8):
        state.append(cpu_backend.buffer_from_pyval(As[x]))
    print("Started")
    out = timed(compiled_computation.execute,state,rate=True,batch=batch)
    out = timed(compiled_computation.execute,out,rate=True,batch=batch)
    print(out[0])

In [21]:
def timed(runer,args,rate=False,batch=1):
    global times
    import time
    t0 = time.time()
    ret = runer(args)
    t1 = time.time()
    if(rate):
        print(f"{batch*times/((t1-t0)):,}")
        return ret
    print(f"{(t1-t0):,}s")
    return ret

In [39]:
# times = 0xffffffff
times = 0xffff

In [40]:
# test computation:
tcb = xc.XlaBuilder("testcomp")
tup = xops.Parameter(tcb, 0, in_tuple_shape)
x = xops.GetTupleElement(tup,0)

const0 = xops.Constant(tcb, special_dtype(times))
y = xops.Gt(const0, x)
test_computation = tcb.Build()

In [41]:
wcb = xc.XlaBuilder("whilecomp")
counter = xops.Parameter(wcb, 0, digit)
records = [counter]
for x in range(64+8):
    records.append(xops.Parameter(wcb, x+1, num))

# mixedTuple = xc.Shape.Tuple(counter,xc.Shape.tuple_shape(x))
res = xops.Tuple(wcb,records)
# res = xops.Tuple(wcb,[counter,b])
xops.While(test_computation, body_computation, res)
wc = wcb.build()

In [42]:
times = 0xffffffff
times = 0xffff
# times = 0x1fffff

In [25]:
455,992,163,188.38776
0x1fffff

893,128,334,421,798.8
0xffffffff

4294967295

In [26]:
use_tpu = False

In [28]:
0xffff

65535

In [43]:
# times = 1000
run(wcb,batch=batch,build=wc)
# print(times*batch)

Evalving
Started
31,417,729.18504972
131,960,495.74651945
65535


## Statistics
#### For times = 0xffffffff and  batch = 100
CPU 835,547,240,504,994.3 749,039,438,889,300.6

GPU Taking Long Time

#### For times = 0xffff and  batch = 10000

GPU 187,438,428.11608002

#### For times = 0xffff and batch = 100

TPU 2,551,479.917635661

Batch = 1000 TPU 19,640,038.66077995
Batch = 10000 TPU 68,014,291.50328211



## TODO
- [ ] Implement getTupleElement Range on JAX